# Нейронные сети. Архитектуры нейронных сетей

## Подготовка данных

В этом практическом заданий мы будем решать задачу классификации цифр на датасете `mnist` с помощью полносвязной и сверточной нейронной сети. Для этого мы будем использовать надстройку над `tensorflow`, которая называется `keras`. Для начала обсудим данные. `mnist` датасет состоит из черно-белых изображений цифр размером $28 \times 28$ пикселей. В данном случае, мы работаем с одним каналом, хотя в случае цветных изображений, общее число каналов равно трем. Загрузим наши данные используя функцию `load_data` объекта `mnist` из модуля `keras.dataset`. Перед выполнением этого задания убедитесь, что ваша версия `tensorflow` >= 1.4.

In [1]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_test.shape, y_test.shape

((10000, 28, 28), (10000,))

Нормализуйте заруженные данные `x_train` и `x_test`. Для этого следует разделить числовое значение каждого пикселя на $255$. Далее, переведите `y_train` и `y_test` в one-hot представление, используя функцию `tf.keras.utils.to_categorical`. Наше первое задание будет заключатся в реализации полносвязной нейронной сети. Поэтому измените размерность тренировочных и тестовых данных с помощью метода `reshape`.
    >> np_vector.shape
    >> (28, 28)
    >> np_vector = np_vector.reshape(28 * 28)
    >> np_vector.shape
    >> (784,)

### *РЕШЕНИЕ*

In [2]:
import numpy as np

x_train = x_train / 255
x_test = x_test / 255

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)


x_train = np.reshape(x_train, (x_train.shape[0], pow(x_train.shape[1],2)))
x_test = np.reshape(x_test, (x_test.shape[0], pow(x_test.shape[1],2)))

x_test.shape, y_test.shape

((10000, 784), (10000, 10))

## Полносвязная нейронная сеть.

В этой части задания вам предлагается реализовать обычную нейронную сеть с использованием последовательной модели `tf.keras.models.Sequential`. Данная модель позволяет добавлять слои при помощи функции встроенной `add`. Наша нейронная сеть будет состоять всего лишь из одного скрытого слоя с количеством нейроннов равным $256$, функцией активации ReLU и с `input_shape=(784,)`, что соответствует количеству нейронов во входном слое нашей нейронной сети. Количество нейроннов в выходном слое равно количеству классов, в качестве функции активации нужно использовать softmax. Не забудьте вызвать `model.compile` после добавления слоев. Используйте в качестве функции потерь `categorical_crossentropy`, оптимизатор `adadelta` и метрику `accuracy`.

### *РЕШЕНИЕ*

In [3]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(256, activation='relu', input_shape=(784,)))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adadelta(),
    metrics=['accuracy']
)

После этого, создайте модель и загрузите веса нейронной сети из файла `neural_networks.h5`. Какое количество настраиваемых параметров содержится в этой нейронной сети. Запишите это число в качестве первого ответа `answer1` на это задание. Оцените качетво на тестовой выборке и запишите это значение с точностью до трех знаков после запятой в переменную `answer2`.

### *РЕШЕНИЕ*

In [4]:
import h5py

def count_params(file_name):
    result = 0
    with h5py.File(file_name, 'r') as file:
        for group in map(lambda x: file[x], file.keys()):
            for keys in map(lambda x: group[x], group.keys()):
                for param in keys.keys():
                    result += np.prod(keys[param].shape)
                    
    return result

answer1 = count_params("../02.ipynb/neural_networks.h5")

model.load_weights('../02.ipynb/neural_networks.h5')
loss, answer2 = model.evaluate(x_test, y_test, verbose=0)

print("answer1:", answer1)
print('answer2:', round(answer2,3))

answer1: 203530
answer2: 0.982


## Сверточная нейронная сеть

Далее, вам предлагается реализовать сверточную нейронную сеть. 

* Размерность входного слоя $(28, 28, 1)$.
* Сверточный слой с $32$ каналами, ядро свертки $3 \times 3$.
* Макспулинг слой $(2,2)$.
* Сверточный слой с $64$ каналами, ядро свертки $3 \times 3$.
* Макспулинг слой $(2,2)$.
* Понижение размерности признаков.
* Полносвязный слой с 64 нейронами
* Выходной слой с количеством нейронов равному количеству классов.

Для этого предлагается использовать следующие классы `Convolution2D`, `MaxPooling2D` и `Flatten` для понижения размерности признаков. Все эти классы как и слой полносвязной нейронной сети `Dense` находятся в `tf.keras.layers`. Используйте ReLU в качестве функции активации во всех слоях, где это потребуется, кроме выходного, в нем по аналогии с прошлым заданием используется softmax. Аналогичная ситуация с функцией `compile` после добавления слоев.

### *РЕШЕНИЕ*

In [5]:
model2 = tf.keras.models.Sequential()

model2.add(tf.keras.layers.Convolution2D(32, kernel_size=(3,3), input_shape=(28,28,1), activation='relu'))

model2.add(tf.keras.layers.MaxPooling2D((2,2)))

model2.add(tf.keras.layers.Convolution2D(64, kernel_size=(3,3), activation='relu'))

model2.add(tf.keras.layers.MaxPooling2D((2,2)))

model2.add(tf.keras.layers.Flatten())

model2.add(tf.keras.layers.Dense(64, activation='relu'))

model2.add(tf.keras.layers.Dense(10, activation='softmax'))

model2.summary()

model2.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adadelta(),
    metrics=['accuracy']
)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense_2 (Dense)             (None, 64)               

Теперь оцените качество получившейся модели на тестовой выборке. Для этого измените размерность `x_train` и `x_test` на размерность входного слоя. Загрузите веса `conv_net.h5`. Запишите количество параметров этой сверточной нейронной сети в `answer3`. Сравните его с количеством параметром в полносвязной нейронной сети, которую мы реализовали ранее. Оценку качества запишите в `answer4` с точностью до 3 трех знаков после запятой.

### *РЕШЕНИЕ*

In [6]:
answer3 = count_params("../02.ipynb/conv_net.h5")
print("answer3:", answer3)

x_train = np.reshape(x_train, (x_train.shape[0], 28, 28, 1))
x_test = np.reshape(x_test, (x_test.shape[0], 28, 28, 1))

model2.load_weights('../02.ipynb/conv_net.h5')
loss, answer4 = model2.evaluate(x_test, y_test, verbose=0)
print('answer4:', round(answer4, 3))

answer3: 121930
answer4: 0.993


# Строка с ответами

In [7]:
output = " nn params {0}\n nn score  {1:.3f}\ncnn params {2}\ncnn score  {3:.3f}"
print(output.format(answer1, answer2, answer3, answer4))

 nn params 203530
 nn score  0.982
cnn params 121930
cnn score  0.993
